<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Nama : Derfansyah Guswiranata Kurnia
- Iddicoding: dgkurnia
- email dicoding: weerakurnia@gmai.com

#Insiasi

In [1]:
#Perpustakaan tahap awal
!pip install selenium beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 33.4 MB/s eta 0:00:00


In [2]:
#Impor Bagian awal
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
#modifikasi percepatan
import asyncio
import aiohttp
from concurrent.futures import ProcessPoolExecutor

In [3]:
#Next Import
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

In [4]:
#Tambahan
from selenium.webdriver.chrome.service import Service

## Sastrawi

In [5]:
#Instalasi sastrawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.6 MB/s eta 0:00:00


In [6]:
#Impor Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
#ke wordcloud

In [7]:
#seksi wordcloud
!pip install wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
# ke NITK

In [8]:
#NITK
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
#impor file tambahan (google drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Perpustakaan lain untuk analisis

In [10]:
#Seksi Io dan lain-lain
import os
from io import StringIO
import concurrent.futures #untuk pemrograman paralel
from concurrent.futures import ThreadPoolExecutor

In [11]:
#Interpolasi untuk perbaikan: Grafik
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d

In [12]:
#Bagian lain
import csv
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score

In [13]:
#pembaruan
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,227 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu

In [14]:
#chromedriver
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 51 not upgraded.
Need to get 30.2 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

### Persiapan untuk analisis

In [15]:
#pandas
import pandas as pd
import time

In [16]:
#Ekstraksi
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer  #sentimen
#pemisahan
from sklearn.model_selection import train_test_split as ttsplit
#ke klasifikasi

In [17]:
#klasifikasi randomforest
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

## Pengambilan ulasan dari situs Tokopedia

In [18]:
#persiapan sesi
session = requests.Session()

In [19]:
#Tahap awal pengambilan
urllist = [
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review"
]

Ada tiga ulasan yang dipakai untuk analisis

In [ ]:
#Persiapan untuk proses pengambilan list: Tahap Extraksi HTM secara asinkronus
async def fetch_html(url):
    response = await asyncio.get_event_loop().run_in_executor(None, session.get, url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.content

# proses 'scraping'
async def review_scrapper(url):
    html_content = await fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')

    # ekstraksi hasil
    results = []  # daftar kosong
    for rev in soup.find_all('div', class_='review-class'):
        teks = rev.find('div', class_='review-content').text.strip()
        results.append(teks)  # penempelan hasil
    return results  # kembalikan hasil

#ke potongan

In [20]:
#persiapan asinkronus untuk pemrosesan url
def process_urls(chunk):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    # persiapan tugas
    tasks = [review_scrapper(url) for url in chunk]
    results = loop.run_until_complete(asyncio.gather(*tasks))

    # pemerataan
    return [item for sublist in results for item in sublist]

#ke fungsi utama

Pengambilan nilai ulasan per potongan

In [21]:
#fungsi pemotongan analisis
def chunk_and_execute(urls, chunk_size=14, output_prefix='intermediate_reviews'):
    output_file = f'{output_prefix}_final.csv'

    # Inspeksi file
    existing_reviews = set()

    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            existing_reviews.update(row['Review'] for row in reader)

    with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Review']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Penulisan baru jika file tidak tersedia
        if not existing_reviews:
            writer.writeheader()

        num_chunks = (len(urls) + chunk_size - 1) // chunk_size

        with ProcessPoolExecutor(max_workers=10) as executor:
            for i in range(num_chunks):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, len(urls))

                chunk_url_list = urls[start_idx:end_idx]

                # Filtrasi URL yang tak diproses
                unprocessed_urls = [url for url in chunk_url_list if url not in existing_reviews]

                if unprocessed_urls:
                    results = executor.submit(process_urls, unprocessed_urls).result()

                    # Write each new review to CSV file
                    for review in results:
                        if review not in existing_reviews:  # Penghindaran duplikasi
                            writer.writerow({'Review': review})
#ke eksekusi

In [ ]:
#penyatuan ke hasil akhir
hslakhir = chunk_and_execute(urllist)
#ke penyetelan data csv

totulas adalah **'total ulasan'** yang dipakai untuk analisis hasi akhir **(hslakhir)**

In [ ]:
#inspeksi hasil akhir
print(hslakhir)

In [ ]:
#persiapan df untuk pembersihan
dfori = pd.DataFrame(hslakhir, columns=['Ulasan'])
dfkopi = dfori.copy() #Jangan pakai data asli
dfkopi.to_csv('tokopedia_reviews.csv', index=False)

data yang dipakai adala **'dfkopi'**

In [ ]:
#Inspeksi dataset
dfinfo = dfkopi.info()
print(dfinfo)

# Tahap awal (pembersihan)

In [ ]:
#persiapan untuk pembersihan


In [ ]:
#Persiapan tahap awal
